In [1]:
import cv2
import numpy as np


def hex_to_bgr(hex_color):
    hex_color = hex_color.lstrip("#")
    rgb = tuple(int(hex_color[i : i + 2], 16) for i in (0, 2, 4))
    return rgb[::-1]


def remove_specific_color(input_path, output_path, target_color, tolerance=30):
    img = cv2.imread(input_path)

    if img is None:
        print(f"Error: Could not read the image at {input_path}")
        return None

    target_color = hex_to_bgr(target_color)

    lower_bound = np.array([max(0, c - tolerance) for c in target_color])
    upper_bound = np.array([min(255, c + tolerance) for c in target_color])
    mask = cv2.inRange(img, lower_bound, upper_bound)

    mask = cv2.bitwise_not(mask)
    result = cv2.bitwise_and(img, img, mask=mask)
    result[mask == 0] = 255

    return result


def increase_contrast(image, alpha=1.5, beta=0):
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)


def make_greys_darker(image, gamma=0.7):
    invGamma = 1.0 / gamma
    table = np.array(
        [((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]
    ).astype("uint8")
    return cv2.LUT(image, table)


def process_image(input_path, output_path, target_color, contrast_alpha=1.5, gamma=0.7):
    # Remove watermark
    result = remove_specific_color(input_path, output_path, target_color)
    if result is None:
        return

    # Increase contrast
    result = increase_contrast(result, alpha=contrast_alpha)

    # Convert to grayscale
    result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

    # Make greys darker
    result = make_greys_darker(result, gamma=gamma)

    # Save the final result
    cv2.imwrite(output_path, result)
    print(f"Processed image saved to {output_path}")


# Use the function
input_image = "1.png"
output_image = "output_1.png"
target_color = "#EBF3F3"  # The color to remove
process_image(input_image, output_image, target_color)

Processed image saved to output_1.png
